# Lab 02 — Terminología y Semántica

## Mini “Terminology Server” dentro de PostgreSQL

**Meta del lab:** comparar analítica **antes vs después** de codificar diagnósticos y laboratorios.


## 0) Conexión a la base de datos

Este notebook asume el mismo setup de Docker del curso (db + jupyter).

- Si no levantaste Docker:
  - Abre una terminal en `labs/lab-2-terminology-semantics/`
  - `docker compose up -d`


In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Database connection string
engine = create_engine('postgresql://uvg_user:uvg_password@db:5432/health_data')

# Simple query to test the foundation layer
try:
    df = pd.read_sql("SELECT 1 as connection_status", engine)
    print("Connection Successful! Your Biomedical Data Stack is ready.")
    print(df)
except Exception as e:
    print(f"Connection Failed: {e}")

Connection Successful! Your Biomedical Data Stack is ready.
   connection_status
0                  1


### Re-crear el modelo de datos de los laboratorios pasados

Vas a ejecutar el scripts SQL `sql/002_ehr_schema.sql` → crea tablas y llenar de datos.

> [!TIP]
> Ejecuta los scripts desde tu terminal (PowerShell) así:
>
> ```powershell
> Get-Content .\sql\002_ehr_schema.sql | docker compose exec -T db psql -U uvg_user -d health_data
> ```


### Ver qué tablas existen


In [5]:
pd.read_sql("""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema='public'
ORDER BY table_name;
""", engine)

,table_schema,table_name
0,public,admissions
1,public,d_labitems
2,public,diagnoses
3,public,labevents
4,public,map_diagnosis_text_to_concept
5,public,map_labitem_to_concept
6,public,patients
7,public,terminology_concepts
8,public,terminology_systems
9,public,v_diagnoses_coded


## 1) BEFORE: Analítica con texto libre (sin terminología)

Primero veremos el problema: _mismo concepto, textos distintos_.

### 1.1 Conteo de diagnósticos por texto (raw)


In [6]:
pd.read_sql("""
SELECT diagnosis_text, COUNT(*) AS n
FROM diagnoses
GROUP BY diagnosis_text
ORDER BY n DESC, diagnosis_text;
""", engine)

,diagnosis_text,n
0,Acute Kidney Injury,1
1,Appendicitis,1
2,Asthma exacerbation,1
3,Chest pain,1
4,Dehydration,1
5,Elective procedure follow-up,1
6,Heart failure,1
7,Hypertension,1
8,Pneumonia,1
9,Sepsis,1


**Observa:** si hay typos o variaciones, aparecen como categorías separadas.

### 1.2 Conteo de labs por label (raw)


In [7]:
pd.read_sql("""
SELECT label, COUNT(*) AS n_items
FROM d_labitems
GROUP BY label
ORDER BY n_items DESC, label;
""", engine)

,label,n_items
0,Creatinine,1
1,Hemoglobin,1
2,Lactate,1
3,Platelets,1
4,White Blood Cells,1


## 2) Crear el “mini terminology server” (tablas)

Vas a ejecutar 3 scripts SQL en orden:

1. `sql/003_terminology_schema.sql` → crea tablas
2. `sql/004_seed_terminology.sql` → llena códigos + sinónimos + mappings
3. `sql/005_apply_mappings.sql` → aplica los mappings al EHR

> [!TIP]
> Ejecuta los scripts desde tu terminal (PowerShell) así:
>
> ```powershell
> Get-Content .\sql\003_terminology_schema.sql | docker compose exec -T db psql -U uvg_user -d health_data
> ```
>
> Repite igual para 004 y 005.


### ✅ Checkpoint A

Cuando termines, corre esta celda y asegúrate que ves tablas `icd10_codes`, `loinc_codes`, `terminology_synonyms`, `diagnosis_text_to_icd10`, `lab_label_to_loinc`.


In [ ]:
pd.read_sql("""
SELECT table_name
FROM information_schema.tables
WHERE table_schema='public'
  AND table_name IN ('icd10_codes','loinc_codes','terminology_synonyms','diagnosis_text_to_icd10','lab_label_to_loinc')
ORDER BY table_name;
""", engine)

## 3) Simular “API”: buscar conceptos por sinónimos

Un terminology server hace cosas como:

- buscar por texto (“wbc” → LOINC 6690-2)
- devolver el concepto “oficial” (código + display + system)

### 3.1 Buscar un término (cambia el texto)


In [ ]:
term = "wbc"  # prueba: 'sepsis', 'hta', 'neumonia', 'creatinine', 'lactate'

pd.read_sql("""
SELECT s.system, s.code, s.synonym, 
       COALESCE(i.display, l.display) AS display
FROM terminology_synonyms s
LEFT JOIN icd10_codes i ON i.code = s.code AND s.system='ICD-10'
LEFT JOIN loinc_codes l ON l.code = s.code AND s.system='LOINC'
WHERE LOWER(s.synonym) LIKE LOWER(%(q)s)
ORDER BY s.system, s.code;
""", engine, params={"q": f"%{term}%"} )

## 4) AFTER: Analítica con códigos (semántica)

### 4.1 Diagnósticos por ICD-10 (ya codificados)


In [ ]:
pd.read_sql("""
SELECT diagnosis_code, diagnosis_display, COUNT(*) AS n
FROM diagnoses
WHERE diagnosis_code IS NOT NULL
GROUP BY diagnosis_code, diagnosis_display
ORDER BY n DESC, diagnosis_code;
""", engine)

### 4.2 ¿Qué diagnósticos quedaron SIN código?

Esto pasa en hospitales reales. Es parte del reto.


In [ ]:
pd.read_sql("""
SELECT diagnosis_text, COUNT(*) AS n
FROM diagnoses
WHERE diagnosis_code IS NULL
GROUP BY diagnosis_text
ORDER BY n DESC, diagnosis_text;
""", engine)

### 4.3 Labs por LOINC (ya codificados)


In [ ]:
pd.read_sql("""
SELECT loinc_code, loinc_display, COUNT(*) AS n_items
FROM d_labitems
WHERE loinc_code IS NOT NULL
GROUP BY loinc_code, loinc_display
ORDER BY n_items DESC, loinc_code;
""", engine)

## 5) Ejercicio: mejora la cobertura de codificación

### Tarea 5.1 — Completa mappings faltantes (diagnósticos)

1. Identifica diagnósticos sin código (celda 4.2)
2. Elige un ICD-10 del mini catálogo (tabla `icd10_codes`)
3. Inserta el mapping en `diagnosis_text_to_icd10`
4. Re-ejecuta `sql/005_apply_mappings.sql`

> [!IMPORTANT]
> No inventes códigos fuera del catálogo del lab. Si hace falta uno, agrégalo a `icd10_codes` primero.

#### Ejemplo (en SQL)

```sql
INSERT INTO diagnosis_text_to_icd10 (diagnosis_text, icd10_code)
VALUES ('Chest pain', 'I10'); -- EJEMPLO MALO: no es correcto clínicamente
```

**Tu trabajo aquí es la parte importante:** ver que _codificar_ requiere criterio y reglas.


### Tarea 5.2 — Completa mappings faltantes (labs)

Haz lo mismo con `d_labitems` → `loinc_codes`.


## 6) Mini-analítica clínica: “Sepsis proxy” (con y sin terminología)

Definición didáctica (NO clínica real):

- WBC > 12 **o** Lactate > 2

### 6.1 Sin terminología: filtrar por labels


In [ ]:
pd.read_sql("""
WITH piv AS (
  SELECT
    le.hadm_id,
    MAX(CASE WHEN li.label = 'White Blood Cells' THEN le.value_num END) AS max_wbc,
    MAX(CASE WHEN li.label = 'Lactate' THEN le.value_num END) AS max_lactate
  FROM labevents le
  JOIN d_labitems li ON li.labitem_id = le.labitem_id
  GROUP BY le.hadm_id
)
SELECT *
FROM piv
WHERE max_wbc > 12 OR max_lactate > 2
ORDER BY hadm_id;
""", engine)

### 6.2 Con terminología: filtrar por LOINC (más robusto)


In [ ]:
pd.read_sql("""
WITH piv AS (
  SELECT
    le.hadm_id,
    MAX(CASE WHEN li.loinc_code = '6690-2' THEN le.value_num END) AS max_wbc,
    MAX(CASE WHEN li.loinc_code = '2524-7' THEN le.value_num END) AS max_lactate
  FROM labevents le
  JOIN d_labitems li ON li.labitem_id = le.labitem_id
  GROUP BY le.hadm_id
)
SELECT *
FROM piv
WHERE max_wbc > 12 OR max_lactate > 2
ORDER BY hadm_id;
""", engine)

## 7) Reflexión (para escribir en el PR)

Responde en el texto del PR:

1. ¿Qué cambió en la analítica al pasar de texto libre → códigos? (1–3 frases)
2. ¿Por qué no basta con “guardar el nombre” del lab o diagnóstico?
3. ¿Qué te faltaría para que esto sea un “terminology server” real (en producción)?

> [!TIP]
> Piensa en: API, versiones, mappings oficiales, validación, gobernanza, logs.
